# ROF Denoising Demo
In this demo we will illustrate the process of image denoising of the ROF model.
$$
\min_{u} \frac{1}{2}\|u-f\|^2+\lambda\sum_{j=1}^n\|(\mathbb{K}u)_j\|
$$

In [1]:
using Pkg, Random, Images, TestImages, LinearAlgebra
using AbstractOperators
using StructuredOptimization

In [5]:
include("../BilevelImagingParameterLearning.jl")
using .BilevelImagingParameterLearning

In [6]:
Random.seed!(123); # For reproductibility

In [7]:
Y = datasetimage("1_smiley_noisy","../datasets/smiley_dataset")
Y[Y .< 0] .= 0. # Make sure pixels are in range
Y[Y .> 1] .= 1.
Gray.(Y)

In [20]:
Xgt = testimage("cameraman")
Y = Xgt .+ sqrt(0.006*norm(Xgt,Inf))*randn(size(Xgt)) # Add noise
Y[Y .< 0] .= 0. # Make sure pixels are in range
Y[Y .> 1] .= 1.
Gray.(Y)
typeof(Y)

Array{Float64,2}

In [12]:
V = Variation(size(Y)) # Define the finite differences gradient operator
U = Variable(size(V,1)...)
lambda = 0.18
Y = float(Y);

In [13]:
@time begin
@minimize ls(-V'*U+Y) + conj(lambda*norm(U,2,1,2)) with ForwardBackward(tol = 1e-3, gamma = 1/8, fast = true) # Minimizing the dual formulation
end

  0.233731 seconds (2.39 k allocations: 106.851 MiB, 10.06% gc time)


((Variable(Float64, (16384, 2)),), 276)

In [10]:
X = Gray.(-V'*(~U)+Y)